# Redes Neuronales con Pytorch I. (torch.nn.Module)

El paquete **torch.nn** de PyTorch contiene multitud de clases que nos permiten crear de una manera intuitiva redes neuronales y a la vez tener un nivel de detalle y control de los componentes de las mismas.

Una de las clases más importantes de **torch.nn** es ***torch.nn.Module*** ya que es muy útil que los modelos que definamos hereden esta clase.   
Para ello es necesario definir el constructor del modelo en *init()* instanciando los componentes que deseemos y sobreescribir el método *forward*, que realiza las operaciones forward del modelo y calcula la salida.  
Con *super.init()* inicializamos la superclase nn.Module. El método forward se ejecuta directamente al llamar al modelo gracias al método call.

En el siguiente ejemplo creamos un modelo basándonos en la clase nn.Module, instanciamos los componentes del modelo usando las clases del paquete torch.nn nn.Linear para la capa lineal y nn.Relu para la función de activacíon ReLU y definimos el flujo de las operaciones en el método forward. 

In [1]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size)  
        self.relu = nn.ReLU()                          
        self.fc2 = nn.Linear(hidden_size, num_classes) 
    
    def forward(self, x):                              
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [2]:
#Creamos una instancia del modelo
modelo1=Net(100,200,10)
#Calculamos la salida con una entrada ejemplo
x=torch.rand(100)
output=modelo1(x)
print(output)

tensor([-0.0804, -0.1233,  0.1534, -0.3510, -0.1157,  0.1984, -0.0913,  0.0540,
        -0.0598,  0.0405], grad_fn=<ViewBackward0>)


Dado que el modelo extiende la clase base nn.Module, es posible usar la gran cantidad de métodos y atributos de esta clase

In [3]:
#Vemos la estructura del modelo
print(modelo1)

Net(
  (fc1): Linear(in_features=100, out_features=200, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=200, out_features=10, bias=True)
)


In [4]:
#Movemos todos los parámetros del modelo a la GPU (Solo equipos dotados y configurados con GPU)
modelo1.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [5]:
#El método named_parameters() devuelve un iterador a los parámetros del modelo, mostrando el nombre del parámetro y el parámetro.
for name, param in modelo1.named_parameters():
  if param.requires_grad:
    if name in ['fc2.weight']:
        print (name, param.data)

fc2.weight tensor([[ 0.0534,  0.0502,  0.0117,  ..., -0.0082,  0.0370,  0.0386],
        [-0.0621,  0.0400,  0.0507,  ..., -0.0266,  0.0499, -0.0456],
        [-0.0034,  0.0228,  0.0232,  ..., -0.0348, -0.0193,  0.0220],
        ...,
        [-0.0411, -0.0395,  0.0591,  ...,  0.0124,  0.0630,  0.0258],
        [-0.0682,  0.0407, -0.0703,  ...,  0.0021,  0.0348,  0.0061],
        [ 0.0478,  0.0483, -0.0497,  ...,  0.0677,  0.0651, -0.0331]])


In [6]:
#El método apply(fn) aplica la función fn recursivamente en cada submódulo del modelo.
#Vamos a crear una función que inicializa los parámetros de las capas lineales y aplicarlo al modelo.

def init_w(m):
  if type(m) == nn.Linear:
    m.weight.data.fill_(1.0)

modelo1.apply(init_w)

#Vemos los parámetros de la segunda capa lineal después de aplicarles la función de inicialización.
for name, param in modelo1.named_parameters():
  if param.requires_grad:
    if name in ['fc2.weight']:
        print (name, param.data)

fc2.weight tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


In [7]:
#El módulo torch.nn también nos ofrece gran variedad de clases que definen las principales arquitecturas de deep learning.
#Un ejemplo es torch.nn.LSTM() que se inicializa con los parámetros (input_size, hidden_size, num_layers).
# Tiene como entrada (input,h_0,c_0) donde la forma de la entrada es (seq_len, batch, input_size).
# La forma de los estados iniciales h_0 y c_0 es (num_layers * num_directions, batch, hidden_size) 

LSTM1 = nn.LSTM(10, 6, 2)
input = torch.randn(5, 2, 10)
h0 = torch.randn(2, 2, 6)
c0 = torch.randn(2, 2, 6)
output, (hn, cn) = LSTM1(input, (h0, c0))

#La salida output tiene la forma (seq_len, batch, num_directions * hidden_size) y contiene la salida h_t para todos los intantes de tiempo.
print(output)

#La salida hn tiene la forma (num_layers * num_directions, batch, hidden_size) y contiene h_t para el último intantes de tiempo en cada capa.
print(hn)

tensor([[[-0.2296, -0.1637, -0.1033, -0.1710,  0.0052,  0.1170],
         [-0.3022, -0.1191,  0.0098, -0.1651, -0.0720,  0.1079]],

        [[-0.1971, -0.1419, -0.0175, -0.2366, -0.0481,  0.1676],
         [-0.2725,  0.0135,  0.0574, -0.0566, -0.1141,  0.2074]],

        [[-0.1964, -0.0802,  0.0521, -0.1611, -0.0998,  0.2026],
         [-0.2584,  0.0119,  0.0519, -0.0776, -0.1616,  0.2448]],

        [[-0.2135, -0.0512,  0.0546, -0.1454, -0.0877,  0.2124],
         [-0.2388, -0.0027,  0.0570, -0.0942, -0.1688,  0.2494]],

        [[-0.1930, -0.0182,  0.0784, -0.1430, -0.1369,  0.2326],
         [-0.2252, -0.0058,  0.0731, -0.0909, -0.1731,  0.2604]]],
       grad_fn=<MkldnnRnnLayerBackward0>)
tensor([[[-0.0729,  0.4497, -0.0678, -0.0376, -0.0460, -0.0429],
         [ 0.0661,  0.1023, -0.1907,  0.0941, -0.2464, -0.1178]],

        [[-0.1930, -0.0182,  0.0784, -0.1430, -0.1369,  0.2326],
         [-0.2252, -0.0058,  0.0731, -0.0909, -0.1731,  0.2604]]],
       grad_fn=<StackBackward0>)
